In [1]:
# Basic import to support interactive widgets in notebooks
import ipywidgets as widgets
from IPython.display import display, Markdown
from ipywidgets import interact, interact_manual

import pandas as pd
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

In [20]:
from secfsdstools.d_container.databagmodel import JoinedDataBag
from secfsdstools.e_filter.joinedfiltering import NoSegmentInfoJoinedFilter

In [4]:
data_dir = "c:/data/sec/automated/_2_all/_1_joined_by_stmt/BS"

In [19]:
bs_bag = JoinedDataBag.load(data_dir)
print(bs_bag.pre_num_df.shape)

(20107441, 17)


In [21]:
no_segments = bs_bag[NoSegmentInfoJoinedFilter()]
print(no_segments.pre_num_df.shape)

(11541310, 17)


In [25]:
value_counts = no_segments.pre_num_df.tag.value_counts()

In [26]:
value_counts.index

Index(['Assets', 'LiabilitiesAndStockholdersEquity', 'StockholdersEquity',
       'RetainedEarningsAccumulatedDeficit', 'CommonStockValue',
       'CashAndCashEquivalentsAtCarryingValue', 'Liabilities',
       'CommonStockSharesAuthorized', 'AssetsCurrent', 'LiabilitiesCurrent',
       ...
       'PaymentsToAcquireRoyaltyInterestsInMiningProperties',
       'DividendsPreferredStockCash',
       'IncreaseDecreaseInAccruedIncomeTaxesPayable',
       'PostconfirmationStockholdersEquity',
       'SharesHeldInEmployeeStockOptionPlanCommittedToBeReleased',
       'ProvisionForLoanLeaseAndOtherLosses',
       'OtherDerivativesNotDesignatedAsHedgingInstrumentsAtFairValueNet',
       'CapitalLeasesNetInvestmentInDirectFinancingLeasesAccumulatedAmortization',
       'OtherComprehensiveIncomeReclassificationOfDefinedBenefitPlansNetGainLossRecognizedInNetPeriodicBenefitCostTax',
       'ConstructionContractorReceivableRetainageAfterYearOne'],
      dtype='object', length=3103)

In [27]:
value_counts['CommonStockSharesOutstanding']

245279

In [28]:
shares_outstanding = no_segments.pre_num_df[no_segments.pre_num_df.tag=='CommonStockSharesOutstanding']

In [29]:
shares_outstanding[:10]

,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote,report,line,stmt,inpth,rfile,plabel,negating
111,0000714562-25-000010,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,11842539.0,None,3,4,BS,1,H,"Common stock, Outstanding shares",0
136,0001493152-25-008460,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,14500685.0,None,3,8,BS,1,H,"Common stock, shares outstanding",0
354,0001393905-25-000073,CommonStockSharesOutstanding,us-gaap/2024,20230930,0,shares,,,7800000.0,None,3,3,BS,1,H,"Common Stock, Shares, Outstanding",0
395,0001558370-25-001826,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,171860000.0,None,3,6,BS,1,H,"Ordinary shares, shares outstanding",0
421,0001628280-25-007202,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,32527244.0,None,4,13,BS,1,H,"Common stock, shares outstanding (in shares)",0
492,0000950170-25-027757,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,27677000.0,None,5,4,BS,1,H,"Common stock, shares outstanding",0
663,0001224608-25-000006,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,101618957.0,None,4,14,BS,1,H,"Common stock, shares outstanding (in shares)",0
665,0001493152-25-011084,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,3652285.0,None,3,13,BS,1,H,"Common stock, shares outstanding",0
678,0001628280-25-012407,CommonStockSharesOutstanding,us-gaap/2024,20241231,0,shares,,,124924185.0,None,4,5,BS,1,H,"Common stock, shares outstanding (in shares)",0
711,0001640334-25-000058,CommonStockSharesOutstanding,us-gaap/2024,20241130,0,shares,,,17452594.0,None,3,4,BS,1,H,"Common stock, shares, outstanding",0
